## No need to run this notebook.

It will take needlessly forever.

In [ ]:
import json
import pathlib

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
from pymedphys.labs.autosegmentation import indexing, filtering

In [ ]:
release_url = 'https://github.com/pymedphys/data/releases/download/structure-dicom'
dicom_zip_url_pattern = f'{release_url}/' + '{dicom_type}.{uid}_Anonymised.zip'
mappings_url = f'{release_url}/mappings.zip'

In [ ]:
data_download_root = pathlib.Path('auto-segmentation-dicom')

In [ ]:
def get_filename_from_url(url):
    filename = url.split('/')[-1]
    
    return filename

In [ ]:
save_filename = data_download_root.joinpath(
    get_filename_from_url(mappings_url))

mappings_paths = pymedphys.zip_data_paths(
    save_filename,
    check_hash=True,
    redownload_on_hash_mismatch=True,
    url=mappings_url
)

In [ ]:
mappings_paths

In [ ]:
data_path_root = mappings_paths[0].parent.parent
data_path_root

In [ ]:
name_mappings_path = data_path_root.joinpath('mappings', 'name_mappings.json')
names_map = filtering.load_names_mapping(name_mappings_path)

In [ ]:
uid_cache_path = data_path_root.joinpath('mappings', 'uid-cache.json')

In [ ]:
with open(uid_cache_path) as f:
    uid_cache = json.load(f)

In [ ]:
(
    ct_image_paths,
    structure_set_paths,
    ct_uid_to_structure_uid,
    structure_uid_to_ct_uids,
) = indexing.read_uid_cache(data_path_root, uid_cache)

In [ ]:
structure_names_mapping_cache_path = data_path_root.joinpath('mappings', 'structure-names-mapping-cache.json')

In [ ]:
with open(structure_names_mapping_cache_path) as f:
    structure_names_cache = json.load(f)
    
structure_names_by_ct_uid = structure_names_cache["structure_names_by_ct_uid"]
structure_names_by_structure_set_uid = structure_names_cache[
    "structure_names_by_structure_set_uid"
]

In [ ]:
uid_to_url = {}

for structure_uid, ct_uids in structure_uid_to_ct_uids.items():
    uid_to_url[structure_uid] = dicom_zip_url_pattern.format(dicom_type='RS', uid=structure_uid)
    
    for ct_uid in ct_uids:
        uid_to_url[ct_uid] = dicom_zip_url_pattern.format(dicom_type='CT', uid=ct_uid)

In [ ]:
def download_uid(data_download_root, uid, uid_to_url, hash_path):
    url = uid_to_url[uid]
    filename = get_filename_from_url(url)
    save_filepath = data_download_root.joinpath('dicom', filename)
    
    pymedphys.zip_data_paths(
        save_filepath,
        check_hash=True,
        redownload_on_hash_mismatch=True,
        delete_when_no_hash_found=True,
        url=url,
        hash_filepath=hash_path
    )

In [ ]:
# Create masks for the following structures, in the following order
structures_to_learn = [
    'lens_left', 'lens_right', 'eye_left', 'eye_right', 'patient']

# Use the following to filter the slices used for training, validation,
# and testing
filters = {
    "study_set_must_have_all_of": structures_to_learn,
    "slice_at_least_one_of": [
        'lens_left', 'lens_right', 'eye_left', 'eye_right'
    ],
    "slice_must_have": ['patient'],
    "slice_cannot_have": []
}